In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from utils import (
    DEGREES_PER_PIXEL,
    TIMESTAMP_IDENT,
    X_PIXELS,
    Y_PIXELS,
    Events,
    extract_gaze_data_between_timestamps_proper,
    get_participant_dominant_eye,
)
import json


from velocityThreshold import detect_fix_ivt, find_sacc_from_fix

# from scipy import ttest_ind

EYE_TRACKER_FOLDER = "eye_tracker_data/"
GAZE_DATA = []
GAZE_DATA_BOOK = []
GAZE_DATA_PAGE = []

# participants that were told that they can change the settings beforehand
participant_ids = [9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
subquery = Events.select().where(Events.participant_id.in_(participant_ids))
avg_saccades = {_id: [] for _id in participant_ids}

print(f"Mean , std , difference")

for participant in participant_ids:
    books = subquery.where(
        (Events.event == "OPEN_BOOK")
        & (Events.new_value != "Chasing Sunsets - ")
        & (Events.participant_id == participant)
    )

    average_length = []
    saccades = []
    for i, book in enumerate(books):
        START_TIME_BOOK = book.time
        book_end = (
            subquery.where(Events.event == "CLOSE_BOOK")
            .where(Events.time > START_TIME_BOOK)
            .get()
        )
        END_TIME_BOOK = book_end.time

        # Get the events that have a timestamp less than the first CLOSE_BOOK event
        events_book = subquery.where(Events.time <= END_TIME_BOOK)

        # Read from participants.json whether or not the participant has low resolution data
        # These we should not divide the timestamp with 1000

        LOW_RES = json.load(open("participants.json", "r"))[f"{book.participant_id}"][
            "low_resolution"
        ]
        if not LOW_RES:
            START_TIME_BOOK /= 1000
        formatted_time = datetime.fromtimestamp(START_TIME_BOOK).strftime(
            "%Y-%m-%d_%H-%M-%S"
        )
        GAZE_FILE = f"{EYE_TRACKER_FOLDER}[{book.participant_id}]-{formatted_time}.json"
        f = open(GAZE_FILE, "r")
        GAZE_DATA_BOOK = json.load(f)
        f.close()

        timestamps = []
        x = []
        y = []

        DOMINANT_EYE = get_participant_dominant_eye(participant)

        # for each packet, plot the gaze point
        for packet in GAZE_DATA_BOOK["data"]:
            if packet[f"{DOMINANT_EYE}_gaze_point_validity"] == 0:
                continue
            x.append(
                (packet[f"{DOMINANT_EYE}_gaze_point_on_display_area"][0] * X_PIXELS)
                # * DEGREES_PER_PIXEL
            )
            y.append(
                (packet[f"{DOMINANT_EYE}_gaze_point_on_display_area"][1] * Y_PIXELS)
                # * DEGREES_PER_PIXEL
            )
            timestamps.append(packet[TIMESTAMP_IDENT])

        df = pd.DataFrame({"x": x, "y": y, "ts": timestamps})
        df = df.sort_values(by="ts")
        df = df.reset_index(drop=True)

        df["x"] = df["x"] * DEGREES_PER_PIXEL
        df["y"] = df["y"] * DEGREES_PER_PIXEL
        df["ts"] = df["ts"] / 1_000_000

        # display(df)
        SACCADIC_THRESHOLD = 80
        # Plot fixations
        fixations, v, labels = detect_fix_ivt(df, sacvel=SACCADIC_THRESHOLD)
        saccades_part = find_sacc_from_fix(fixations)
        average_length.append(np.mean(saccades_part["len"]))

    print(
        f"{participant} {average_length[0]} {average_length[1]} {np.std(average_length)} {average_length[0] - average_length[1]}"
    )

df = pd.DataFrame(avg_saccades)
# print the mean saccade length for each participant
display(df.mean())

    # Do a t-test to see if the mean saccade length is significantly different between the two books
    # # Perform t-test
    # t_statistic, p_value = ttest_ind(saccades[0]['len'], saccades[0]['len'])

    # # Print the results
    # print(f"T-Statistic: {t_statistic}")
    # print(f"P-Value: {p_value}")

Mean , std , difference


/Users/larsrefsgaard/Documents/Git/flow/apps/backend/velocityThreshold.py:62: RuntimeWarning: invalid value encountered in scalar divide
  rho = cov[0,1] / (sx*sy)
/Users/larsrefsgaard/Documents/Git/flow/apps/backend/velocityThreshold.py:62: RuntimeWarning: invalid value encountered in scalar divide
  rho = cov[0,1] / (sx*sy)


9 0.021104795102164457 0.024027883023778892 0.0014615439608072177 -0.0029230879216144354


/Users/larsrefsgaard/Documents/Git/flow/apps/backend/velocityThreshold.py:62: RuntimeWarning: invalid value encountered in scalar divide
  rho = cov[0,1] / (sx*sy)
/Users/larsrefsgaard/Documents/Git/flow/apps/backend/velocityThreshold.py:62: RuntimeWarning: invalid value encountered in scalar divide
  rho = cov[0,1] / (sx*sy)


10 0.0267587375299098 0.022483576177019585 0.0021375806764451083 0.004275161352890217


...

/Users/larsrefsgaard/Documents/Git/flow/apps/backend/velocityThreshold.py:62: RuntimeWarning: invalid value encountered in scalar divide
  rho = cov[0,1] / (sx*sy)
/Users/larsrefsgaard/Documents/Git/flow/apps/backend/velocityThreshold.py:62: RuntimeWarning: invalid value encountered in scalar divide
  rho = cov[0,1] / (sx*sy)


20 0.03097984567814561 0.026800844827314194 0.002089500425415709 0.004179000850831418


/Users/larsrefsgaard/Documents/Git/flow/apps/backend/velocityThreshold.py:62: RuntimeWarning: invalid value encountered in scalar divide
  rho = cov[0,1] / (sx*sy)
/Users/larsrefsgaard/Documents/Git/flow/apps/backend/velocityThreshold.py:62: RuntimeWarning: invalid value encountered in scalar divide
  rho = cov[0,1] / (sx*sy)


21 0.021054970600167925 0.019579225956918078 0.0007378723216249234 0.0014757446432498468


9     0.022566
10    0.024621
11    0.024005
12    0.030484
13    0.022509
14    0.024724
15    0.022152
16    0.022700
17    0.019577
18    0.026974
19    0.017803
20    0.028890
21    0.020317
dtype: float64